In [6]:
import os
import sqlite3
from openai import OpenAI
from vanna.openai import OpenAI_Chat
from vanna.chromadb import ChromaDB_VectorStore

# Componentes

In [8]:
class MyVanna( ChromaDB_VectorStore, OpenAI_Chat):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        OpenAI_Chat.__init__(self, config=config)
        self.client = OpenAI(api_key=config['openai']['api_key'])

# Inicialização do Vanna

In [9]:
vn = MyVanna(
    config={
        'openai': {
            'api_key': os.getenv('OPENAI_API_KEY'),
            'model': 'gpt-3.5-turbo'
        },
        'chroma': {
            'persist_directory': 'chroma-db'
        }
    }
)

In [10]:
vn.connect_to_sqlite( url = 'db_olist.sqlite')

# Vanna Client

## Treinamento Vanna com DLL

In [19]:
consulta_dll = """
                SELECT
                    name, sql
                FROM
                    sqlite_master
                WHERE type = 'table'
                ORDER BY
                    name;
"""
ddl_olist = vn.run_sql(consulta_dll)

In [21]:
treinamento = 0
for _, row in ddl_olist.iterrows():
    tbl, ddl = row['name'], row['sql']
    if isinstance (ddl, str) and ddl.strip():
        vn.train(ddl = ddl)
        treinamento += 1
        print(f'DDL registrado: {ddl}')
print(f'[ok] Tabelas treinadas: {treinamento}')

Adding ddl: CREATE TABLE customer( 
    customer_id                 TEXT,
    customer_unique_id          TEXT,
    customer_zip_code_prefix    INTEGER,  
    customer_city               TEXT,
    customer_state              TEXT
)


C:\Users\gbnob\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:14<00:00, 5.72MiB/s]


DDL registrado: CREATE TABLE customer( 
    customer_id                 TEXT,
    customer_unique_id          TEXT,
    customer_zip_code_prefix    INTEGER,  
    customer_city               TEXT,
    customer_state              TEXT
)
Adding ddl: CREATE TABLE geolocation( 
    geolocation_zip_code_prefix    INTEGER,
    geolocation_lat                REAL,
    geolocation_lng                REAL,
    geolocation_city               TEXT,
    geolocation_state              TEXT
)
DDL registrado: CREATE TABLE geolocation( 
    geolocation_zip_code_prefix    INTEGER,
    geolocation_lat                REAL,
    geolocation_lng                REAL,
    geolocation_city               TEXT,
    geolocation_state              TEXT
)
Adding ddl: CREATE TABLE order_items( 
    order_id               TEXT,
    order_item_id          INTEGER,
    product_id             TEXT,
    seller_id              TEXT,
    shipping_limit_date    TEXT,
    price                  REAL,
    freight_value       

In [22]:
vn.get_training_data()

,id,question,content,training_data_type
0,cbfd11d3-9114-58e7-b3a6-111d46188360-ddl,None,CREATE TABLE customer( \n customer_id ...,ddl
1,ed7d3c96-8bfb-55da-b6af-d55b6bce6418-ddl,None,CREATE TABLE geolocation( \n geolocation_zi...,ddl
2,8c734f07-49b1-5e1a-984b-b1ef3e10fac8-ddl,None,CREATE TABLE order_items( \n order_id ...,ddl
3,f01472c1-90e1-5877-95a4-9a2c6b701173-ddl,None,CREATE TABLE order_payments( \n order_id ...,ddl
4,9bab394c-c8ed-5e42-97a8-f741e5e21c85-ddl,None,CREATE TABLE order_review_backup (\n\treview_i...,ddl
5,5ebeea56-9273-5381-9222-852f2ba4a4c3-ddl,None,CREATE TABLE order_review_shorts (\n\treview_i...,ddl
6,75b917f3-829f-5022-bccf-211fc9e14e12-ddl,None,CREATE TABLE order_reviews( \n review_id ...,ddl
7,c5903464-875c-5b19-bd93-000e79217a4d-ddl,None,CREATE TABLE orders( \n order_id ...,ddl
8,d56e42a5-118f-5846-93a7-f5b8521ec7a7-ddl,None,CREATE TABLE product_category_name( \n prod...,ddl
9,691e6822-1c45-5a1e-b146-1395ca68d408-ddl,None,CREATE TABLE products( \n product_id ...,ddl


## Treinamento Vanna via Perguntas e SQL

In [27]:
queries = [
    {
        "question": (
            "Calcule o GMV (soma do price) e o nº de pedidos por mês de compra. "
            "Saídas: year_month, orders_cnt, gmv"
        ),
        "sql": """
            SELECT
                STRFTIME('%Y-%m', o.order_purchase_timestamp) AS year_month,
                COUNT(DISTINCT o.order_id) AS orders_cnt,
                SUM(price) AS gmv
            FROM orders o
            LEFT JOIN order_items i
                ON i.order_id = o.order_id
            GROUP BY year_month
            ORDER BY year_month;
        """,
    },
    {
        "question": (
            "Liste as 5 categorias com mais itens vendidos (quantidade de linhas em order_items). "
            "Saídas: product_category_name, itens_vendidos"
        ),
        "sql": """
            SELECT
                p.product_category_name,
                COUNT(*) AS itens_vendidos
            FROM order_items i
            LEFT JOIN products p
                ON p.product_id = i.product_id
            GROUP BY p.product_category_name
            ORDER BY itens_vendidos DESC
            LIMIT 5;
        """,
    },
    {
        "question": (
            "Calcule o valor médio de pagamento por tipo de pagamento. "
            "Saídas: payment_type, avg_payment_value"
        ),
        "sql": """
            SELECT
                op.payment_type,
                AVG(op.payment_value) AS avg_payment_value
            FROM order_payments op
            GROUP BY payment_type
            ORDER BY avg_payment_value DESC;
        """,
    },
    {
        "question": (
            "Tempo médio de entrega (em dias) por estado do cliente. "
            "Saídas: customer_state, avg_delivery_days"
        ),
        "sql": """
            SELECT
                c.customer_state,
                AVG(
                    julianday(o.order_delivered_customer_date)
                    - julianday(o.order_purchase_timestamp)
                ) AS avg_delivery_days
            FROM orders o
            INNER JOIN customer c
                ON c.customer_id = o.customer_id
            WHERE o.order_delivered_customer_date IS NOT NULL
            GROUP BY c.customer_state
            ORDER BY avg_delivery_days;
        """,
    },
    {
        "question": (
            "Taxa de cancelamento por mês (considerando order_status = 'canceled'). "
            "Saídas: year_month, orders_cnt, canceled_cnt, cancel_rate"
        ),
        "sql": """
            SELECT
                STRFTIME('%Y-%m', o.order_purchase_timestamp) AS year_month,
                COUNT(*) AS orders_cnt,
                SUM(CASE WHEN o.order_status = 'canceled' THEN 1 ELSE 0 END) AS canceled_cnt,
                CAST(
                    SUM(CASE WHEN o.order_status = 'canceled' THEN 1 ELSE 0 END) AS REAL
                ) / COUNT(*) AS cancel_rate
            FROM orders o
            GROUP BY year_month
            ORDER BY year_month;
        """,
    },
    {
        "question": (
            "Para cada seller, calcule o GMV total (soma do price) e o nº de pedidos únicos. "
            "Saídas: seller_id, orders_cnt, gmv"
        ),
        "sql": """
            SELECT
                oi.seller_id,
                COUNT(DISTINCT o.order_id) AS orders_cnt,
                ROUND(SUM(oi.price), 2) AS gmv
            FROM orders o
            LEFT JOIN order_items oi
                ON oi.order_id = o.order_id
            GROUP BY oi.seller_id
            ORDER BY gmv DESC, orders_cnt DESC;
        """,
    },
    {
        "question": (
            "Top 3 sellers por GMV em cada mês (ranking mensal usando janela). "
            "Saídas: year_month, seller_id, gmv, rn"
        ),
        "sql": """
            WITH seller_month AS (
                SELECT
                    STRFTIME('%Y-%m', o.order_purchase_timestamp) AS year_month,
                    oi.seller_id,
                    SUM(oi.price) AS gmv
                FROM orders o
                LEFT JOIN order_items oi
                    ON oi.order_id = o.order_id
                GROUP BY year_month, oi.seller_id
            ),
            ranked AS (
                SELECT
                    year_month,
                    seller_id,
                    gmv,
                    ROW_NUMBER() OVER (
                        PARTITION BY year_month
                        ORDER BY gmv DESC
                    ) AS rn
                FROM seller_month
            )
            SELECT
                year_month, seller_id, gmv, rn
            FROM ranked
            WHERE rn <= 3
            ORDER BY year_month, rn, gmv DESC;
        """,
    },
    {
        "question": (
            "GMV mensal e GMV acumulado móvel de 3 meses (janela). "
            "Saídas: year_month, gmv_mth, gmv_3m_rolling"
        ),
        "sql": """
            WITH monthly AS (
                SELECT
                    STRFTIME('%Y-%m', o.order_purchase_timestamp) AS year_month,
                    SUM(i.price) AS gmv_mth
                FROM orders o
                JOIN order_items i
                    ON i.order_id = o.order_id
                GROUP BY year_month
            )
            SELECT
                year_month,
                gmv_mth,
                SUM(gmv_mth) OVER (
                    ORDER BY year_month
                    ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
                ) AS gmv_3m_rolling
            FROM monthly
            ORDER BY year_month;
        """,
    },
    {
        "question": (
            "Atraso vs estimativa: '%' de pedidos entregues após order_estimated_delivery_date, por UF do cliente (top 10 por volume). "
            "Saídas: customer_state, late_delivery_rate, orders_cnt"
        ),
        "sql": """
            WITH late_table AS (
                SELECT
                    g.geolocation_state AS customer_state,
                    COUNT(o.order_id) AS orders_cnt,
                    SUM(
                        CASE
                            WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date
                            THEN 1 ELSE 0
                        END
                    ) AS late_delivery
                FROM orders o
                INNER JOIN customer c
                    ON c.customer_id = o.customer_id
                INNER JOIN geolocation g
                    ON g.geolocation_zip_code_prefix = c.customer_zip_code_prefix
                GROUP BY g.geolocation_state
            )
            SELECT
                customer_state,
                orders_cnt,
                CAST(late_delivery AS REAL) / orders_cnt AS late_delivery_rate
            FROM late_table
            ORDER BY orders_cnt DESC
            -- LIMIT 10  -- descomente para manter apenas o top 10 por volume
            ;
        """,
    },
]

In [28]:
for q in queries:
    vn.train(question=q["question"], sql=q["sql"])

## Treinamento Vanna via Documentação

In [25]:
documentations = [
    # --- Descrições simples ---
    "Tabela `orders` contém pedidos, status, timestamps de compra e entrega.",
    "Tabela `customers` contém informações dos clientes e CEP.",
    "Tabela `order_items` contém os itens de cada pedido, com preço e frete.",
    "Tabela `products` contém categorias e informações de produtos.",
    "Tabela `sellers` contém vendedores, localização e identificação.",
    "Tabela `payments` contém pagamentos de cada pedido.",
    "Tabela `reviews` contém avaliações de clientes sobre pedidos.",

    # --- Relações entre tabelas da Olist ---
    "A tabela `orders` se conecta à tabela `customers` pela coluna `customer_id` usando INNER JOIN.",
    "A tabela `orders` se conecta à tabela `order_items` pela coluna `order_id` usando LEFT JOIN (um pedido pode ter mais de um item).",
    "A tabela `orders` se conecta à tabela `order_payments` pela coluna `order_id` usando LEFT JOIN (nem todo pedido tem pagamento completo registrado).",
    "A tabela `orders` se conecta à tabela `order_reviews` pela coluna `order_id` usando LEFT JOIN (nem todo pedido possui review).",
    "A tabela `order_items` se conecta à tabela `products` pela coluna `product_id` usando LEFT JOIN (pois alguns produtos podem não estar catalogados).",
    "A tabela `order_items` se conecta à tabela `sellers` pela coluna `seller_id` usando INNER JOIN.",
    "A tabela `sellers` se conecta à tabela `geolocation` pela coluna `zip_code_prefix` usando INNER JOIN.",
    "A tabela `customers` se conecta à tabela `geolocation` pela coluna `zip_code_prefix` usando INNER JOIN.",

    # --- Boas práticas de data ---
    "Sempre utilize a função strftime no formato '%Y-%m' para gerar ano e mês no SQLite.",
    "Nunca utilize '%y-%m' ou outros formatos de strftime, apenas '%Y-%m'."
]

In [26]:
# Exemplo de uso:
for doc in documentations:
    vn.train(documentation=doc)

Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....


## Treinamento Vanna Prompt

In [30]:
vn.config["sql_prompt_preamble"] = """Você é um especialista em SQL para SQLite.Sempre que precisar agrupar ou formatar datas por ano e mês, 
                                        use: strftime('%Y-%m', coluna_data)Nunca use '%y-%m' ou outros formatos.Sua resposta deve ser **somente a query SQL válida**
                                        sem comentários nem explicações.Nunca responda com texto, apenas SQL executável. Não invente colunas. Não use outras formas
                                        de join.Sempre coloque na esquerda a tabela de referência para fazer os joins."""

# Geração da consulta SQL

In [31]:
pergunta = "Liste os 10 maiores consumidores e o total em valor e a quantidade de produtos que ele comprou"

sql = vn.generate_sql(question = pergunta)

SQL Prompt: [{'role': 'system', 'content': "You are a SQLite expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \nCREATE TABLE order_items( \n    order_id               TEXT,\n    order_item_id          INTEGER,\n    product_id             TEXT,\n    seller_id              TEXT,\n    shipping_limit_date    TEXT,\n    price                  REAL,\n    freight_value          REAL\n)\n\nCREATE TABLE products( \n    product_id                    TEXT,\n    product_category_name         TEXT,\n    product_name_lenght           REAL,\n    product_description_lenght    REAL,\n    product_photos_qty            REAL,\n    product_weight_g              REAL,\n    product_length_cm             REAL,\n    product_height_cm             REAL,\n    product_width_cm              REAL\n)\n\nCREATE TABLE product_category_name( \n    product_category_name       

In [35]:
pergunta = "Faça um rank dos melhores vendedores do mês 08/2018, por valor de venda naquele mês. Acrescente o total de itens vendidos para cada vendedor."

sql = vn.generate_sql(question = pergunta)

SQL Prompt: [{'role': 'system', 'content': "You are a SQLite expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \nCREATE TABLE order_reviews( \n    review_id                  TEXT,\n    order_id                   TEXT,\n    review_score               INTEGER,\n    review_comment_title       TEXT,\n    review_comment_message     TEXT,\n    review_creation_date       TEXT,\n    review_answer_timestamp    TEXT\n)\n\nCREATE TABLE order_items( \n    order_id               TEXT,\n    order_item_id          INTEGER,\n    product_id             TEXT,\n    seller_id              TEXT,\n    shipping_limit_date    TEXT,\n    price                  REAL,\n    freight_value          REAL\n)\n\nCREATE TABLE orders( \n    order_id                         TEXT,\n    customer_id                      TEXT,\n    order_status                     TEXT,\n    order_

In [37]:
df = vn.run_sql(sql)

In [45]:
df.head(10)

,seller_id,total_sales,total_items_sold,sales_rank
0,1025f0e2d44d7041d6cf58b6550e0bfa,16791.60,138,1
1,53243585a1d6dc2643021fd1853d8905,16438.00,80,2
2,4869f7a5dfa277a7dca6462dcf3b52b2,14429.40,86,3
3,6560211a19b47992c3666cc44a7e94c0,13354.00,204,4
4,6061155addc1e54b4cfb51c1c2a32ad8,11203.50,23,5
5,f7ba60f8c3f99e7ee4042fdef03b70c4,10841.00,31,6
6,7d13fca15225358621be4086e1eb0964,9812.04,50,7
7,fa1c13f2614d7b5c4749cbc52fecda94,8714.80,32,8
8,04308b1ee57b6625f47df1d56f00eedf,8500.40,16,9
9,240b9776d844d37535668549a396af32,8245.25,25,10


In [41]:
from matplotlib import pyplot as plt